In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
region_data = pd.read_csv('./Pricing Data Case Study/regiondata.csv')
region_data.rename(columns = {'ISO-2':'Country_Code'},inplace= True)

## CoffeeBucks payment traffic data

In [3]:
payment_traffic = pd.read_csv('./Pricing Data Case Study/samplepaymenttraffic.csv')

# merge region into dataset
payment_traffic = pd.merge(payment_traffic, region_data, left_on='Issuer_Country', right_on='Country_Code')


# create Transaction_Regionality
payment_traffic.rename(columns = {'Region':'IssuerRegion'} , inplace= True)
payment_traffic['Transaction_Regionality'] = np.where(payment_traffic['Acquirer_Country'] == payment_traffic['Issuer_Country'],'DOMESTIC','INTER_REGIONAL' )

# blend rate is 2.4%
payment_traffic['Fee_In_Blend_Rate'] = payment_traffic['Settled_Amount_in_Processing_Currency'] * 0.024
payment_traffic['Fee_In_Blend_Rate'] = payment_traffic['Fee_In_Blend_Rate'].map('{:,.2f}'.format)
payment_traffic["Fee_In_Blend_Rate"] = [float(str(i).replace(",", "")) for i in payment_traffic["Fee_In_Blend_Rate"]]
payment_traffic['Fee_In_Blend_Rate'] = payment_traffic['Fee_In_Blend_Rate'].astype(float)
#markup rate is  0.20%
payment_traffic['markup_fee'] = payment_traffic['Settled_Amount_in_Processing_Currency'] * 0.002
payment_traffic['markup_fee'] = payment_traffic['markup_fee'].map('{:,.2f}'.format)
payment_traffic["markup_fee"] = [float(str(i).replace(",", "")) for i in payment_traffic["markup_fee"]]
payment_traffic['markup_fee'] = payment_traffic['markup_fee'].astype(float)
# scheme rate is determined as 0.99%
payment_traffic['Scheme_fee_rate'] =np.where(payment_traffic['Transaction_Regionality'] =='DOMESTIC',0.001, 0.01)
payment_traffic['Scheme_fee'] = payment_traffic['Settled_Amount_in_Processing_Currency'] * payment_traffic['Scheme_fee_rate']
payment_traffic['Scheme_fee'] = payment_traffic['Scheme_fee'].map('{:,.3f}'.format)
payment_traffic["Scheme_fee"] = [float(str(i).replace(",", "")) for i in payment_traffic["Scheme_fee"]]
payment_traffic['Scheme_fee'] = payment_traffic['Scheme_fee'].astype(float)

# Divide data set into visa and mc
Visa_payment = payment_traffic[payment_traffic['Card_Scheme'] == 'visa']
MC_payment = payment_traffic[payment_traffic['Card_Scheme'] == 'mc']


In [24]:
len(payment_traffic[payment_traffic['Number_of_Settled_Transactions'] > 50]) / len(payment_traffic)

0.035357437165224555

## First price proposal 

In [104]:
ContAuth_visa = payment_traffic[(payment_traffic['Shopper_Interaction'] == 'ContAuth') & (payment_traffic['Card_Scheme'] == 'visa')]
ContAuth_mc = payment_traffic[(payment_traffic['Shopper_Interaction'] == 'ContAuth') & (payment_traffic['Card_Scheme'] == 'mc')]
Ecommerce_visa = payment_traffic[(payment_traffic['Shopper_Interaction'] == 'Ecommerce') & (payment_traffic['Card_Scheme'] == 'visa')]
Ecommerce_mc = payment_traffic[(payment_traffic['Shopper_Interaction'] == 'Ecommerce') & (payment_traffic['Card_Scheme'] == 'mc')]

,Acquirer_Country,Issuer_Country,Card_Scheme,Payment_Method_Variant,Shopper_Interaction,3DS_Attempt,Date,Processing_Currency,Settled_Amount_in_Processing_Currency,Number_of_Settled_Transactions,Country,Country_Code,IssuerRegion,Transaction_Regionality,Fee_In_Blend_Rate,markup_fee,Scheme_fee_rate,Scheme_fee
5,SG,PL,visa,visastandarddebit,ContAuth,False,4/1/2020,GBP,67.42,1,Poland,PL,EUROPE,INTER_REGIONAL,1.62,0.13,0.01,0.674
7,SG,PL,visa,visastandardcredit,ContAuth,False,4/1/2020,AUD,137.95,1,Poland,PL,EUROPE,INTER_REGIONAL,3.31,0.28,0.01,1.379
13,SG,PL,visa,visasuperpremiumdebit,ContAuth,False,4/1/2020,PLN,382.54,1,Poland,PL,EUROPE,INTER_REGIONAL,9.18,0.77,0.01,3.825
14,SG,PL,visa,visacommercialcredit,ContAuth,False,4/1/2020,PLN,208.40,1,Poland,PL,EUROPE,INTER_REGIONAL,5.00,0.42,0.01,2.084
17,SG,PL,visa,visapremiumcredit,ContAuth,False,4/1/2020,PLN,413.93,2,Poland,PL,EUROPE,INTER_REGIONAL,9.93,0.83,0.01,4.139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38827,SG,CG,visa,visastandarddebit,ContAuth,False,4/27/2020,EUR,88.05,1,Congo (Brazzaville),CG,CEMEA,INTER_REGIONAL,2.11,0.18,0.01,0.880
38828,SG,CG,visa,visastandarddebit,ContAuth,False,4/29/2020,EUR,52.40,1,Congo (Brazzaville),CG,CEMEA,INTER_REGIONAL,1.26,0.10,0.01,0.524
38829,SG,BJ,visa,visastandarddebit,ContAuth,False,4/19/2020,EUR,95.60,1,Benin,BJ,CEMEA,INTER_REGIONAL,2.29,0.19,0.01,0.956
38830,SG,BJ,visa,visastandarddebit,ContAuth,False,4/20/2020,EUR,51.09,1,Benin,BJ,CEMEA,INTER_REGIONAL,1.23,0.10,0.01,0.511


In [ ]:
ContAuth_visa = pd.merge(Ecommerce_visa,Visa_Payment_Method, how = 'left', on=['Payment_Method_Variant','Transaction_Regionality'] )
ContAuth_visa.rename(columns = {'Fee_Rate':'Inter_Fee_Rate'},inplace= True)
# new 
ContAuth_visa['Inter_Fee_Rate'] = ContAuth_visa['Inter_Fee_Rate'] * 0.2
ContAuth_visa['Fee_In_Inter_Rate'] = ContAuth_visa['Settled_Amount_in_Processing_Currency'] * ContAuth_visa['Inter_Fee_Rate']
ContAuth_visa['Fee_In_Inter_Rate'] = ContAuth_visa['Fee_In_Inter_Rate'].map('{:,.2f}'.format)
ContAuth_visa['Fee_In_Inter_Rate'] = [float(str(i).replace(",", "")) for i in ContAuth_visa['Fee_In_Inter_Rate']]
ContAuth_visa['Fee_In_Inter_Rate'] = ContAuth_visa['Fee_In_Inter_Rate'].astype(float)

ContAuth_visa['Fee_In_InterPlusPlus'] = Ecommerce_visa['Fee_In_Inter_Rate'] + Ecommerce_visa['markup_fee'] + Ecommerce_visa['Scheme_fee']

In [ ]:
Ecommerce_visa = pd.merge(Ecommerce_visa,Visa_Payment_Method, how = 'left', on=['Payment_Method_Variant','Transaction_Regionality'] )
Ecommerce_visa.rename(columns = {'Fee_Rate':'Inter_Fee_Rate'},inplace= True)
Ecommerce_visa['Fee_In_Inter_Rate'] = Ecommerce_visa['Settled_Amount_in_Processing_Currency'] * Ecommerce_visa['Inter_Fee_Rate']
Ecommerce_visa['Fee_In_Inter_Rate'] = Ecommerce_visa['Fee_In_Inter_Rate'].map('{:,.2f}'.format)
Ecommerce_visa['Fee_In_Inter_Rate'] = [float(str(i).replace(",", "")) for i in Ecommerce_visa['Fee_In_Inter_Rate']]
Ecommerce_visa['Fee_In_Inter_Rate'] = Ecommerce_visa['Fee_In_Inter_Rate'].astype(float)

Ecommerce_visa['Fee_In_InterPlusPlus'] = Ecommerce_visa['Fee_In_Inter_Rate'] + Ecommerce_visa['markup_fee'] + Ecommerce_visa['Scheme_fee']

In [25]:
# read MC, VISA interchange fee rate
MC_Payment_Method = pd.read_csv('./Pricing Data Case Study/MC_Payment_Method.csv')
Visa_Payment_Method = pd.read_csv('./Pricing Data Case Study/Visa_Payment_Method.csv')

In [84]:
# merge propose interchange fee to clent's payment data
Visa_payment_v = pd.merge(Visa_payment,Visa_Payment_Method, how = 'left', on=['Payment_Method_Variant','Transaction_Regionality'] )
Visa_payment_v.rename(columns = {'Fee_Rate':'Inter_Fee_Rate'},inplace= True)
Visa_payment_v['Fee_In_Inter_Rate'] = Visa_payment_v['Settled_Amount_in_Processing_Currency'] * Visa_payment_v['Inter_Fee_Rate']
Visa_payment_v['Fee_In_Inter_Rate'] = Visa_payment_v['Fee_In_Inter_Rate'].map('{:,.2f}'.format)
Visa_payment_v['Fee_In_Inter_Rate'] = [float(str(i).replace(",", "")) for i in Visa_payment_v['Fee_In_Inter_Rate']]
Visa_payment_v['Fee_In_Inter_Rate'] = Visa_payment_v['Fee_In_Inter_Rate'].astype(float)

Visa_payment_v['Fee_In_InterPlusPlus'] = Visa_payment_v['Fee_In_Inter_Rate'] + Visa_payment_v['markup_fee'] + Visa_payment_v['Scheme_fee']

In [111]:
# new price proposal  
Visa_payment_v['Fee_In_Inter_Rate_New'] = np.where(Visa_payment_v['Shopper_Interaction'] == 'ContAuth', Visa_payment_v['Fee_In_Inter_Rate'] * 0.2, Visa_payment_v['Fee_In_Inter_Rate'])
Visa_payment_v['Fee_In_InterPlusPlus_New'] = Visa_payment_v['Fee_In_Inter_Rate_New'] + Visa_payment_v['markup_fee'] + Visa_payment_v['Scheme_fee']

In [113]:
Visa_payment_v.groupby(['Payment_Method_Variant','Transaction_Regionality'])[['Fee_In_Blend_Rate','Fee_In_InterPlusPlus','Fee_In_InterPlusPlus_New']].apply(sum)

Fee_In_Blend_Rate  \
Payment_Method_Variant           Transaction_Regionality                      
visacommercialcredit             DOMESTIC                            177.84   
                                 INTER_REGIONAL                     4444.42   
visacommercialdebit              INTER_REGIONAL                    11407.13   
visacommercialpremiumcredit      DOMESTIC                            144.98   
                                 INTER_REGIONAL                      559.05   
visacommercialpremiumdebit       DOMESTIC                            442.70   
                                 INTER_REGIONAL                        6.48   
visacommercialsuperpremiumcredit INTER_REGIONAL                     1001.60   
visacorporatecredit              DOMESTIC                            130.00   
                                 INTER_REGIONAL                      711.53   
visacorporatedebit               INTER_REGIONAL                      299.81   
visafleetcredit                  INTER_REGIONAL                      132.60   
visapremiumcredit                DOMESTIC                         145366.80   
                                 INTER_REGIONAL                    69416.59   
visapremiumdebit                 DOMESTIC                         135843.08   
                                 INTER_REGIONAL                     8083.11   
visapurchasingcredit             DOMESTIC                             23.72   
                                 INTER_REGIONAL                      400.03   
visapurchasingdebit              INTER_REGIONAL                        5.81   
visastandardcredit               DOMESTIC                           2341.61   
                                 INTER_REGIONAL                   157325.35   
visastandarddebit                DOMESTIC                           1464.92   
                                 INTER_REGIONAL                   525706.79   
visasuperpremiumcredit           DOMESTIC                           3690.85   
                                 INTER_REGIONAL                     2844.16   
visasuperpremiumdebit            DOMESTIC                             57.01   
                                 INTER_REGIONAL                     4214.92   

                                                          Fee_In_InterPlusPlus  \
Payment_Method_Variant           Transaction_Regionality                         
visacommercialcredit             DOMESTIC                              138.570   
                                 INTER_REGIONAL                       5833.386   
visacommercialdebit              INTER_REGIONAL                      14971.816   
visacommercialpremiumcredit      DOMESTIC                              114.173   
                                 INTER_REGIONAL                          0.000   
visacommercialpremiumdebit       DOMESTIC                              379.933   
                                 INTER_REGIONAL                          0.000   
visacommercialsuperpremiumcredit INTER_REGIONAL                          0.000   
visacorporatecredit              DOMESTIC                              101.292   
                                 INTER_REGIONAL                        933.831   
visacorporatedebit               INTER_REGIONAL                          0.000   
visafleetcredit                  INTER_REGIONAL                          0.000   
visapremiumcredit                DOMESTIC                           116899.099   
                                 INTER_REGIONAL                      86770.434   
visapremiumdebit                 DOMESTIC                            99618.253   
                                 INTER_REGIONAL                      10003.319   
visapurchasingcredit             DOMESTIC                               20.758   
                                 INTER_REGIONAL                          0.000   
visapurchasingdebit              INTER_REGIONAL                          0.000   
visastandardcredit               DOMESTIC   

In [99]:
# merge propose interchange fee to clent's payment data
MC_payment_v = pd.merge(MC_payment,MC_Payment_Method, how = 'left', on=['Payment_Method_Variant','Transaction_Regionality'] )
MC_payment_v.rename(columns = {'Fee_Rate':'Inter_Fee_Rate'},inplace= True)
MC_payment_v['Fee_In_Inter_Rate'] = MC_payment_v['Settled_Amount_in_Processing_Currency'] * MC_payment_v['Inter_Fee_Rate']
MC_payment_v['Fee_In_Inter_Rate'] = MC_payment_v['Fee_In_Inter_Rate'].map('{:,.2f}'.format)
MC_payment_v['Fee_In_Inter_Rate'] = [float(str(i).replace(",", "")) for i in MC_payment_v['Fee_In_Inter_Rate']]
MC_payment_v['Fee_In_Inter_Rate'] = MC_payment_v['Fee_In_Inter_Rate'].astype(float)

MC_payment_v['Fee_In_InterPlusPlus'] = MC_payment_v['Fee_In_Inter_Rate'] + MC_payment_v['markup_fee'] + MC_payment_v['Scheme_fee']

In [114]:
MC_payment_v['Fee_In_Inter_Rate_New'] = np.where(MC_payment_v['Shopper_Interaction'] == 'ContAuth', MC_payment_v['Fee_In_Inter_Rate'] * 0.2, MC_payment_v['Fee_In_Inter_Rate'])
MC_payment_v['Fee_In_InterPlusPlus_New'] = MC_payment_v['Fee_In_Inter_Rate_New'] + MC_payment_v['markup_fee'] + MC_payment_v['Scheme_fee']

In [115]:
Visa_payment_v.to_csv('payment_in_visa.csv', encoding='utf-8', index=False)
MC_payment_v.to_csv('payment_in_mc.csv', encoding='utf-8', index=False)

In [192]:
payment_traffic.to_csv('payment_traffic_new.csv', encoding='utf-8', index=False)

## Adyen interchange and scheme fee data

In [5]:
inter_rate = pd.read_csv('./Pricing Data Case Study/sampleinterchangerates.csv')
# inter_rate['value_per_payment'] = inter_rate['Total_Value_of_Payments_(Local Currency)'] / inter_rate['Number_of_Payments']
inter_rate['fee_rate'] = inter_rate['Total_Fee_(Local Currency)'] / inter_rate['Total_Value_of_Payments_(Local Currency)']
inter_rate['fee_rate'] = inter_rate['fee_rate'].map('{:,.4f}'.format)
inter_rate['fee_rate'] = inter_rate['fee_rate'].astype(float)

inter_rate['Booking_Date']= inter_rate['Booking_Date'].apply(pd.to_datetime)

# get rid of anomaly
inter_rate = inter_rate[inter_rate['Total_Fee_(Local Currency)'].gt(0)]

# merge dataset
inter_rate = pd.merge(left = inter_rate, right = region_data, how = 'left', left_on = 'Acquirer_Country_Code', right_on = 'Country_Code')
inter_rate.rename(columns = {'Region':'AcquirerRegion'} , inplace= True)

In [6]:
inter_rate

,Acquirer,Acquirer_Country_Code,Transaction_Regionality,Shopper_Interaction,Payment_Method_Variant,Funding_Source,Fee_Category,3DS_Attempt,Booking_Date,Local_Currency,Number_of_Payments,Total_Value_of_Payments_(Local Currency),Total_Fee_(Local Currency),fee_rate,Country,Country_Code,AcquirerRegion
0,AdyenMC_EU,DE,INTER_REGIONAL,ContAuth,mcpremiumcredit,CREDIT,INTERCHANGE,False,2020-04-01,EUR,340,6089.40,88.30,0.0145,Germany,DE,EUROPE
1,AdyenVisa_EU,NL,INTER_REGIONAL,ContAuth,visastandarddebit,DEBIT,SCHEMEFEE,False,2020-04-01,EUR,990,27185.40,289.30,0.0106,Netherlands,NL,EUROPE
2,AdyenVisa_EU,NL,INTER_REGIONAL,ContAuth,visastandarddebit,DEBIT,SCHEMEFEE,False,2020-04-01,EUR,131,5795.44,53.67,0.0093,Netherlands,NL,EUROPE
3,AdyenMC_EU,NL,INTER_REGIONAL,ContAuth,mcsuperpremiumcredit,CREDIT,INTERCHANGE,False,2020-04-01,EUR,624,9609.60,139.34,0.0145,Netherlands,NL,EUROPE
4,AdyenMC_EU,NL,INTER_REGIONAL,ContAuth,mcsuperpremiumcredit,CREDIT,SCHEMEFEE,False,2020-04-01,EUR,169,7861.88,68.00,0.0086,Netherlands,NL,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,AdyenVisa_EU,NL,INTRA_REGIONAL,Ecommerce,visastandarddebit,DEBIT,INTERCHANGE,False,2020-04-30,EUR,204,5750.76,8.63,0.0015,Netherlands,NL,EUROPE
16196,AdyenMC_SG,SG,INTRA_REGIONAL,ContAuth,mccommercialcredit,CREDIT,INTERCHANGE,False,2020-04-30,SGD,300,3330.00,64.94,0.0195,Singapore,SG,APAC
16197,AdyenVisa_EU,NL,INTER_REGIONAL,Ecommerce,visasuperpremiumcredit,CREDIT,INTERCHANGE,False,2020-04-30,EUR,285,5947.95,86.25,0.0145,Netherlands,NL,EUROPE
16198,AdyenVisa_EU,NL,INTRA_REGIONAL,ContAuth,visastandarddebit,DEBIT,INTERCHANGE,False,2020-04-30,EUR,294,5503.68,8.26,0.0015,Netherlands,NL,EUROPE


In [7]:
inter_rate.fee_rate.describe()

count    16200.000000
mean         0.010388
std          0.005888
min          0.000000
25%          0.007000
50%          0.011000
75%          0.014500
max          0.058900
Name: fee_rate, dtype: float64

In [202]:
#inter_rate.to_csv('inter_rate_new.csv', encoding='utf-8', index=False)

In [13]:
INTERCHANGE_SG = inter_rate[(inter_rate['Fee_Category'] == 'INTERCHANGE') & (inter_rate['Acquirer_Country_Code'] == 'SG') ]

In [14]:
SCHEMEFEE_SG  = inter_rate[(inter_rate['Fee_Category'] == 'SCHEMEFEE') & (inter_rate['Acquirer_Country_Code'] == 'SG')]

In [15]:
INTERCHANGE_SG.to_csv('interchange_sg.csv', encoding='utf-8', index=False)
SCHEMEFEE_SG.to_csv('scheme_sg.csv', encoding='utf-8', index=False)

In [80]:
INTERCHANGE_SG.fee_rate.describe()

count    914.000000
mean       0.015257
std        0.004297
min        0.004500
25%        0.012000
50%        0.016550
75%        0.019300
max        0.023000
Name: fee_rate, dtype: float64

In [74]:
InterFee_gp = INTERCHANGE_SG.groupby(['Payment_Method_Variant','Transaction_Regionality'])['fee_rate'].agg(['median','mean']).reset_index()

In [75]:
InterFee_gp.to_csv('inter_fee_sg.csv', encoding='utf-8', index=False)

In [173]:
INTERCHANGE_SG.groupby(['Funding_Source'])['fee_rate'].agg(['median','mean'])

,median,mean
Funding_Source,,
CREDIT,0.018,0.015655
DEBIT,0.015,0.014559


In [176]:
INTERCHANGE_SG.groupby(['Transaction_Regionality','Funding_Source'])['fee_rate'].agg(['median','mean'])

median      mean
Transaction_Regionality Funding_Source                  
DOMESTIC                CREDIT          0.0170  0.015197
                        DEBIT           0.0164  0.013965
INTER_REGIONAL          CREDIT          0.0180  0.017347
                        DEBIT           0.0155  0.016184
INTRA_REGIONAL          CREDIT          0.0180  0.015406
                        DEBIT           0.0130  0.014065